**420-A58-SF - Algorithmes d'apprentissage non supervisé - Hiver 2023 - Spécialisation technique en Intelligence Artificielle**<br/>
MIT License - Copyright (c) 2023 Mikaël Swawola
<br/>
# Examen #2
<br/>

# À propos de l'examen

* **Date et heure**: Mercredi 17 mai 2023 de 18h30 à 22h00 (3.5 heures) ;
* **Pondération**: 30% ;
* **Tous les documents sont autorisés** ;
* **Il s'agit d'un travail individuel. Toute collaboration (avec quiconque!) et plagiat sont strictement interdits. Le non respect de cette consigne entraînera une note de 0 à l'examen** ;
* L'examen est noté sur 100 points ;
* **<span style='color: red'>Conformément aux directives de la fédération des Cégeps, le professeur se réserve le droit de demander aux élèves d'activer leurs caméras et microphones afin de s'assurer du respect des conditions d'examen. Tout étudiant qui choisit de refuser cette mesure se place dans une situation équivalente à une non présentation à l’examen et s’expose donc à un échec. </span>**

---
# Avant de commencer

* Cet examen couvre les parties 1 et 2 du cours *420-A58-SF - Apprentisage non supervisé*. Des connaissances des *cours 420-A55-SF - Analyse exploratoire des données* et *420-A56-SF - Transformation de données* relatives à l'apprentissage non supervisé peuvent être requises (ex. ACP et TF-IDF).
* Les jeux de données sont considérés comme préparés. Ils ne contiennent ni valeurs abérrantes, ni données manquantes.
* Vous devrez rendre ce fichier complété de **vos réponses et de votre code** sur la plateforme Lea à la fin de l'examen.
* Renommez le présent fichier en incluant vos noms et prénoms. Exemple: **Examen2_&lt;nom&gt;_&lt;prenom&gt;.ipynb**
* <strong style='color: red'>Lisez bien les questions</strong> et **bon succès à tous !**

---
# Description de la problématique

Cet examen consiste à réaliser l'analyse d'un corpus de tweets à l'aide des différentes méthodes d'apprentissage non supervisé étudiées en cours. Le corpus est constitué de tweets collectés depuis Twitter le lundi 11 Août 2020. Deux fichiers sont fournis dans le répertoire `data`:
* `tweets_raw.csv`: champ `text` des tweets non transformés
* `tweets_prc.csv`: champ `text` des tweets préparés pour l'analyse (casse, suppression des caractère spéciaux et numériques, suppression des stopwords, URLs, stemming, etc...)

Un fichier `helpers.py` est disponible si besoin.

**Les parties 1,2,3,4 et 5 doivent être réalisées dans l'ordre. La partie 6 requiert la réalisation de la partie 1.**

---
# 1 - Chargement des données (5 points)

### Question 1-1 - Chargez le jeu de données comprenant les <u>tweets préparés</u> dans une trame de donnée nommée `corpus`. Combien de tweets comprend le jeu de données ? (2.5 points)

### ----- Votre réponse ci-dessous -----

In [ ]:
# Maximum 5 lignes de code

import pandas as pd
NAME = "tweets_prc"
corpus = pd.read_csv(f'../../data/{NAME}.csv', index_col=[0])
len(corpus)

### Question 1-2 - À vue d'œil, le jeu de données contient-il des tweets sur différents sujets ? Citez deux sujets ou thématiques éventuellement présentes dans ce corpus. (2.5 points)

Indice: vous pouvez vous aider du fichier de tweets non transformé pour répondre à cette question.

### ----- Votre réponse ci-dessous -----

In [ ]:
# Question rédactionnelle

---
# 2 - Extraction de la représentation numérique des tweets (10 points)

### Question 2-1 - Pourquoi la représentation TF-IDF est-elle préférable dans le cas présent à la représentation par sacs de mots ? Justifiez et illustrez votre réponse à l'aide d'un exemple. (5 points)

### ----- Votre réponse ci-dessous -----

In [ ]:
# Question rédactionnelle

### Question 2-2 -  Réalisez l'extraction des représentations TF-IDF correspondant aux tweets préparés et stockez les dans une variable nommée `X`. Quelle est la taille du dictionnaire (nombre de mots) ? (5 points)

Indice: la fonction `TfidfVectorizer` de la librairie scikit-learn vous sera utile
[sklearn.feature_extraction.text.TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

### ----- Votre réponse ci-dessous -----

In [ ]:
# Maximum 5 lignes de code

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus['text'])
len(vectorizer.get_feature_names_out())

---
# 3 - Recherche de thématiques (25 points)

### Question 3-1 - À l'aide de la technique du partitionnement hiérarchique, recherchez les sujets ou thématiques présentes dans le corpus. Donnez une estimation du nombre de sujets. (20 points)

Indice: il est inutile ici de comparer les différentes méthodes de lien: le lien de Ward sera suffisant.

### ----- Votre réponse ci-dessous -----

In [ ]:
# Maximum 10 lignes de code

from sklearn.cluster import AgglomerativeClustering
from helpers import *

X_ = X.toarray()
model = AgglomerativeClustering(distance_threshold=0, n_clusters=None, metric="euclidean", linkage="ward")
model.fit(X_)

plot_dendrogram(model)

### Question 3-2 - Visualisez les partitions obtenues à l'aide d'une technique de réduction de dimensions. Cette visualisation est-elle utile ? Justifiez votre réponse. (5 points)

### ----- Votre réponse ci-dessous -----

In [ ]:
# Maximum 10 lignes de code

import seaborn as sns
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(X_)
df_pca = pca.transform(X_)

model = AgglomerativeClustering(distance_threshold=None, n_clusters=6, metric="euclidean", linkage="ward")
model.fit(X_)
ax = sns.scatterplot(x=df_pca[:,0], y=df_pca[:,1], hue=model.labels_, palette="Set1")

In [ ]:
pca.explained_variance_ratio_

---
# 4 - Annotation des partitions (20 points)

Nous allons maintenant annoter les différents clusters découverts précédemment. Pour se faire, nous allons réaliser une **recherche d'itemsets fréquents par l'algorithme Apriori. Un choix purement arbitraire sera que le label (ou l'étiquette) donnée au cluster correspondra à la paire d'items de ce cluster ayant le support le plus élevé**.

### Question 4-1 - On décide d'appliquer le modèle market basket aux tweets du corpus. À quels éléments du jeu de données correspondent les baskets et les items ? (2.5 points)

### ----- Votre réponse ci-dessous -----

In [ ]:
# Question rédactionnelle

#Baskets = tweets
#Items = mots

### Question 4-2 - Exécutez l'algorithme Apriori sur la partition d'indice 0. Quelles paires de mots les plus fréquentes correspondent à cette partition  ? (10 points)

Remarque: référez-vous à la documentation [apriori](http://rasbt.github.io/mlxtend/user_guide/frequent_patterns/apriori/) pour savoir comment trouver uniquement les paires fréquentes.

In [ ]:
model.labels_
c0 = corpus.loc[model.labels_ == 0]
c0
transactions = list(c0['text'].apply(lambda x:x.split(' ')))
transactions

In [ ]:
#!pip install mlxtend
# Maximum 15 lignes de code

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

# Cette fonction sera utile pour isoler les clusters
def isolate_cluster(corpus, model, cluster = 0):
    c0 = corpus.loc[model.labels_ == cluster]
    transactions = list(c0['text'].apply(lambda x:x.split(' ')))
    te = TransactionEncoder()
    te_data = te.fit(transactions).transform(transactions)
    
    return pd.DataFrame(te_data,columns=te.columns_)

# Cluster 0
df = isolate_cluster(corpus, model, 0)
freq0 = apriori(df, min_support=0.1, use_colnames=True)
freq0['length'] = freq0['itemsets'].apply(lambda x: len(x))
freq0[freq0['length'] == 2].sort_values(by='support', ascending=False).head(5)

### Question 4-3 - Répétez la question précédente sur les autres partitions.  (5 points)

Indice: Vous pouvez créer une fonction pour éviter la répétition de code 

### ----- Votre réponse ci-dessous -----

In [ ]:
# Maximum 20 lignes de code

# Cluster 1
df = isolate_cluster(corpus, model, 1)
freq1 = apriori(df, min_support=0.1, use_colnames=True)
freq1['length'] = freq1['itemsets'].apply(lambda x: len(x))
freq1[freq1['length'] == 2].sort_values(by='support', ascending=False).head(5)

In [ ]:
# Cluster 2
df = isolate_cluster(corpus, model, 2)
freq2 = apriori(df, min_support=0.02, use_colnames=True)
freq2['length'] = freq2['itemsets'].apply(lambda x: len(x))
freq2[freq2['length'] == 2].sort_values(by='support', ascending=False).head(5)

In [ ]:
# Cluster 3
df = isolate_cluster(corpus, model, 3)
freq3 = apriori(df, min_support=0.05, use_colnames=True)
freq3['length'] = freq3['itemsets'].apply(lambda x: len(x))
freq3[freq3['length'] == 2].sort_values(by='support', ascending=False).head(5)

In [ ]:
# Cluster 4
df = isolate_cluster(corpus, model, 4)
freq4 = apriori(df, min_support=0.05, use_colnames=True)
freq4['length'] = freq4['itemsets'].apply(lambda x: len(x))
freq4[freq4['length'] == 2].sort_values(by='support', ascending=False).head(5)

In [ ]:
# Cluster 5
df = isolate_cluster(corpus, model, 5)
freq5 = apriori(df, min_support=0.05, use_colnames=True)
freq5['length'] = freq5['itemsets'].apply(lambda x: len(x))
freq5[freq5['length'] == 2].sort_values(by='support', ascending=False).head(5)

### Question 4-4 - Visualisez de nouveau les partitions en affichant cette fois-ci les labels identifiés à la question précédente. (2.5 points)

### ----- Votre réponse ci-dessous -----

In [ ]:
# Maximum 5 lignes de code

labels = ['flyer-canadien', 'federer-wimbledon', 'misc', 'veganfood', 'tornado-twister', 'elon musk']
labels = [labels[x] for x in model.labels_]

ax = sns.scatterplot(x=df_pca[:,0], y=df_pca[:,1], hue=labels, palette="Set1")

---
# 5 - Règles d'association (20 points)

### Question 5-1 - L'une des partition semble montrer une association inusitée. Laquelle ? Quelles sont les règles d'association correspondantes ? Donnez leurs indices de confiance. (15 points)

### ----- Votre réponse ci-dessous -----

In [ ]:
# Maximum 2 lignes de code
df_ar1 = association_rules(freq1, metric = "support", min_threshold = 0.3)
df_ar1.head(10)

### Question 5-2 - Faites une recherche de cette association sur Internet. Trouvez-vous un article d'actualité datant de 2020 démontrant que cette association existe réellement ? Si oui, fournissez le lien vers l'article. (5 points)

### ----- Votre réponse ci-dessous -----

In [ ]:
# Réponse rédactionnelle

https://thenextweb.com/neural/2020/08/14/this-ai-simulation-of-roger-federer-can-help-him-overcome-his-wimbledon-shame/

---
# 6 - Recherche de tweets similaires (20 points)

Soit le tweet (authentique) suivant:
    
**@elonmusk #BREAKING: Elon Musk, CEO of Tesla &amp; SpaceX Do Not Sleep!**

et sa version préparée:

**break elon musk ceo tesla spacex sleep**

### Question 6-1 - Quelle technique de similarité est la mieux adaptée à la recherche de tweets similaire ? Justifiez votre réponse. (5 points)

In [ ]:
# Réponse rédactionnelle

### Question 6-2 - À l'aide d'une méthode de recherche vue en cours, identifier les 5 tweets du corpus les plus similaires à ce nouveau tweet ? Donnez les index et les distances/similarités. (10 points)

### ----- Votre réponse ci-dessous -----

In [ ]:
# Maximum 10 lignes de code

from sklearn.neighbors import NearestNeighbors

In [ ]:
new_tweet = "break elon musk ceo tesla amp spacex sleep"

In [ ]:
new_row = pd.DataFrame([{'text': new_tweet}])
new_corpus = pd.concat([corpus, new_row], ignore_index=True)
X_new = vectorizer.fit_transform(new_corpus['text']) # Refaire la transformation dans le cas de nouveaux mots

In [ ]:
neigh = NearestNeighbors(n_neighbors=1, metric="cosine")
neigh.fit(X_new[:-1,:]) # On retire la nouvelle entrée pour le pas l'avoir dans les résultats
nearest = neigh.kneighbors(X_new[-1,:], 5)
nearest

### Question 6-3 - Afficher les 5 tweets identifiés précédemment en prenant le texte non préparé. (5 points)

### ----- Votre réponse ci-dessous -----

In [ ]:
# Maximum 4 lignes de code

NAME = "tweets_raw"
pd.set_option('display.max_colwidth', None)
corpus = pd.read_csv(f'../../data/{NAME}.csv', index_col=[0])
corpus.loc[nearest[1][0]]

---
## Fin de l'examen #2